In [1]:
#install  google-api-python-client

In [4]:
import store_to_vdb

# from apiclient.discovery import build
# from googleapiclient.discovery import build

In [20]:
captions = store_to_vdb.playlist_to_captions("PLUl4u3cNGP60uVBMaoNERc6knT_MgPKS0")
len(captions)

22

In [21]:
captions = store_to_vdb.captions_processing(captions)
print(captions[0]['video_captions'][0]['text'])
for i in range(len(captions)):
    print(len(captions[i]['video_captions']))

The following content is provided under a Creative Commons license.
1034
897
1015
843
836
810
1065
793
723
913
975
673
729
687
864
677
748
774
797
700
918
579


In [22]:
print(len(captions[5]['video_captions']))
embeds_test = store_to_vdb.captions_to_embeddings([captions[5]])


810


In [4]:
# import time
# import cohere


# def captions_to_embeddings(videos_captions):
#     api_key = 'SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT'
#     # Create and retrieve a Cohere API key from os.cohere.ai
#     co = cohere.Client(api_key)
#     all_embeds = []
#     no_texts = 0
#     for captions in videos_captions :
#         embeds = []
#         texts = []
#         for caption in captions['video_captions'] :
#             dic = {}
#             dic['start'] = caption['start']
#             dic['video_id'] = captions['video_id']
#             texts.append(caption['text'])
#             dic['embedding'] = []  
#             embeds.append(dic)
#         if(len(texts) > 1600):
#             #call on video captions embedding
# #             video_embeddings = 
#             pass
#         else:
#             if((no_texts+len(texts)) > 1600 ):
#                 no_texts = 0 
#                 print("sleep")
#                 time.sleep(60)
#             no_texts += len(texts)
#             video_embeddings = co.embed(texts,model="small").embeddings
#             print(len(texts))
        
#         for i in range(len(embeds)):
#             embeds[i]['embedding'] =  video_embeddings[i]
        
#         all_embeds.append(embeds)
          
#     return all_embeds


In [29]:
len(captions)

22

In [36]:
emds = store_to_vdb.captions_to_embeddings(captions[16:])
len(emds)

6

In [37]:
import numpy as np
import pinecone
PIENCONE_API_KEY = "3d2006de-95b3-4e7d-9ec1-54133c34001e"
pinecone.init(PIENCONE_API_KEY, environment='us-west1-gcp')
index_name = 'first-index'
index = pinecone.Index(index_name)

last_id =  index.describe_index_stats()['total_vector_count']+1

In [38]:
last_id

21681

In [39]:
import numpy as np
import pinecone
# import cohere
# co = cohere.Client("SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT")

# query = "hello world"

# index = pinecone.Index("first-index")
# # create the query embedding
# xq = co.embed(
#     texts=[query],
#     model='small',
#     truncate='LEFT'
# ).embeddings[0]

# example = [[ {"start":6 , "video_id":"C1lhuz6pZC0" , "embedding":xq,"text": query } ]]




def Piencone_indexing(collection):    #    -> [[{}]]  -> [[ {start:--- , video_id:--- , embedding:--- }   ,]]
    
    PIENCONE_API_KEY = "3d2006de-95b3-4e7d-9ec1-54133c34001e"
    
    pinecone.init(PIENCONE_API_KEY, environment='us-west1-gcp')
    index_name = 'first-index'
    print('============1===============')
    
    shape = np.array(collection[0][0]["embedding"]).shape       # (1024,)

    # if the index does not exist, we create it
    if index_name not in pinecone.list_indexes():
        pinecone.create_index(
            index_name,
            dimension=shape[0],
            metric='dotproduct'
        )
    # connect to index
    index = pinecone.Index(index_name)
    print('============2===============')
    
    last_id =  index.describe_index_stats()['total_vector_count']+1
    for x in range(len(collection)):
        list_dic_data = collection[x]
        print(len(collection[x]))
        num_embeds = len(list_dic_data)
        print(shape)

        batch_size = 128

        ids = [str(i) for i in range( last_id , last_id+ num_embeds + 1)]
        last_id = last_id + num_embeds + 1
        # create list of metadata dictionaries
        embeds = []
        meta_data = []
        
        for dic_data in list_dic_data:
            
            meta = {}
            for key in dic_data.keys():
                if key == 'embedding':
                    embeds.append(dic_data[key])
                else:
                    meta[key] = dic_data[key]
            meta_data.append(meta)

        print(meta_data[0]["video_id"] , " ",len(meta_data))


        # create list of (id, vector, metadata) tuples to be upserted
        to_upsert = list(zip(ids, embeds, meta_data))
#         to_upsert = list(zip(ids, embeds))
        for i in range(0, shape[0]+1, batch_size):
            i_end = min(i+batch_size, shape[0])
            a = to_upsert[i:i_end]
            if(len(a) == 0):
                print("empty")
                break

            index.upsert(vectors=a)

        # let's view the index statistics
        print(index.describe_index_stats())

    return None

In [40]:
Piencone_indexing(emds)

============1===============
============2===============
748
(1024,)
WW3ZJHPwvyg   748
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 22349}},
 'total_vector_count': 22349}
774
(1024,)
a1ZCeFpeW0o   774
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 23123}},
 'total_vector_count': 23123}
797
(1024,)
X-ix97pw0xY   797
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 23920}},
 'total_vector_count': 23920}
700
(1024,)
mc1y8m9-hOM   700
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 24620}},
 'total_vector_count': 24620}
918
(1024,)
OYcdw5vOgIc   918
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 25538}},
 'total_vector_count': 25538}
579
(1024,)
lWW54ts9Ubo   579
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 26117}},
 'total_vector_count': 26117}


In [43]:
import cohere
co = cohere.Client("SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT")

query = "population bias"

index = pinecone.Index("first-index")
# create the query embedding
xq = co.embed(
    texts=[query],
    model='small',
    truncate='LEFT'
).embeddings

print(np.array(xq).shape)

# query, returning the top 5 most similar results
res = index.query(xq, top_k=5, include_metadata=True)

(1, 1024)


In [44]:
res

{'matches': [{'id': '20316',
              'metadata': {'start': 509.33,
                           'text': 'Maybe this population is not the '
                                   'population in the world.',
                           'video_id': 'bFZ-0FH5hfs'},
              'score': 1403.53088,
              'sparseValues': {},
              'values': []},
             {'id': '7152',
              'metadata': {'start': 2002.77,
                           'text': 'It is true from the Census Bureau data.',
                           'video_id': 'K2SC-WPdT6k'},
              'score': 1306.06445,
              'sparseValues': {},
              'values': []},
             {'id': '8652',
              'metadata': {'start': 4103.84,
                           'text': 'Here what I am assuming is that my '
                                   'population is homogeneous.',
                           'video_id': 'VPZD_aij8H0'},
              'score': 1287.48157,
              'sparseValues': {},
 

In [ ]:
def captions_to_embeddings(videos_captions):
    api_key = 'SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT'
    # Create and retrieve a Cohere API key from os.cohere.ai
    co = cohere.Client(api_key)
    all_embeds = []
    for captions in videos_captions :
        embeds = []
        texts = []
        video_embeddings = []
        caps = captions['video_captions']
        for i in range(len(caps)):
            texts.append(caps[i]['text'])
            dic['start'] = caps[i]['start']
            dic['video_id'] = caps[i]['video_id']
            embeds.append(dic)
            if((i+1)%16 == 0):
                video_embeddings + = co.embed(texts,truncate='RIGHT'model="small").embeddings
                text = []
        if(len(texts) > 0):
            video_embeddings + = co.embed(texts,truncate='RIGHT'model="small").embeddings
            
        for i in range(len(embeds)):
            embeds[i]['embedding'] =  video_embeddings[i]
        all_embeds.append(embeds)    